In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

## What this Program Does

< Insert a flow diagram on the entire workflow >

< Table about what each combination of Hyper parameters resulted in >

1. Loading and preprocessing the dataset.
2. Extracting handcrafted features (e.g., color histograms, texture descriptors).
3. Training and evaluating ML classifiers (SVM and Decision Tree).

## Hyperparameters :-

1. What dimension to resize all the images into :-  ( resized_width, resized_height )

In [ ]:
# Hyper-parameters
resized_width, resized_hieght = 64, 64


In [ ]:
# Load Dataset
def load_images_from_folder(folder, label):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = cv2.imread(img_path)
        if img is not None:
            # Resize to a fixed size
            img = cv2.resize(img, (resized_width, resized_hieght))  
            images.append(img)
            labels.append(label)
    return images, labels

# Define Paths
mask_folder = "../data/with_mask_without_mask_classification/with_mask"
no_mask_folder = "../data/with_mask_without_mask_classification/without_mask"

# Load Images
mask_images, mask_labels = load_images_from_folder(mask_folder, 1)
no_mask_images, no_mask_labels = load_images_from_folder(no_mask_folder, 0)

## Perform EDA on both with_mask and no_mask ImageSet

In [ ]:
# eda


[array([[[201, 186, 190],
         [186, 171, 175],
         [177, 161, 165],
         ...,
         [ 51,  49,  55],
         [ 62,  60,  66],
         [ 68,  66,  71]],
 
        [[197, 182, 186],
         [181, 166, 170],
         [171, 156, 160],
         ...,
         [ 50,  48,  54],
         [ 45,  43,  49],
         [ 50,  48,  54]],
 
        [[187, 173, 177],
         [173, 159, 165],
         [166, 152, 158],
         ...,
         [ 34,  33,  39],
         [ 38,  36,  41],
         [ 46,  44,  50]],
 
        ...,
 
        [[246, 252, 251],
         [249, 251, 251],
         [251, 251, 251],
         ...,
         [ 43,  39,  39],
         [ 55,  50,  49],
         [ 65,  60,  59]],
 
        [[247, 253, 252],
         [250, 252, 252],
         [252, 252, 252],
         ...,
         [ 50,  42,  40],
         [ 57,  50,  47],
         [ 68,  61,  58]],
 
        [[247, 253, 252],
         [250, 252, 252],
         [251, 252, 252],
         ...,
         [ 58,  49,  47],
  

In [3]:
# Combine Data
X = np.array(mask_images + no_mask_images)
y = np.array(mask_labels + no_mask_labels)

# Convert to grayscale for feature extraction
X_gray = np.array([cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) for img in X])

In [4]:
# Extract Features - Histogram of Oriented Gradients (HOG)
from skimage.feature import hog

def extract_hog_features(images):
    features = []
    for img in images:
        feature, _ = hog(img, orientations=8, pixels_per_cell=(8, 8),
                        cells_per_block=(2, 2), visualize=True)
        features.append(feature)
    return np.array(features)

X_features = extract_hog_features(X_gray)

# Split Data
X_train, X_test, y_train, y_test = train_test_split(X_features, y, test_size=0.2, random_state=42)

In [5]:
# Train and Evaluate ML Classifiers

# SVM Classifier
svm_clf = SVC(kernel='linear')
svm_clf.fit(X_train, y_train)
svm_pred = svm_clf.predict(X_test)
svm_acc = accuracy_score(y_test, svm_pred)

# Decision Tree Classifier
dt_clf = DecisionTreeClassifier()
dt_clf.fit(X_train, y_train)
dt_pred = dt_clf.predict(X_test)
dt_acc = accuracy_score(y_test, dt_pred)

print(f"SVM Accuracy: {svm_acc:.2f}")
print(f"Decision Tree Accuracy: {dt_acc:.2f}")

SVM Accuracy: 0.92
Decision Tree Accuracy: 0.76
